In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/240000-household-electricity-consumption-records/household_power_consumption.csv


In [2]:
data = pd.read_csv('/kaggle/input/240000-household-electricity-consumption-records/household_power_consumption.csv')

In [3]:
data.head()

,index,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,0,1/1/07,0:00:00,2.58,0.136,241.97,10.6,0,0,0.0
1,1,1/1/07,0:01:00,2.552,0.1,241.75,10.4,0,0,0.0
2,2,1/1/07,0:02:00,2.55,0.1,241.64,10.4,0,0,0.0
3,3,1/1/07,0:03:00,2.55,0.1,241.71,10.4,0,0,0.0
4,4,1/1/07,0:04:00,2.554,0.1,241.98,10.4,0,0,0.0


In [4]:
data = pd.read_csv('/kaggle/input/240000-household-electricity-consumption-records/household_power_consumption.csv', parse_dates=['Date'], index_col='Date')
data['Global_active_power'] = data['Global_active_power'].replace('?',np.nan)

data['Global_active_power'] = data['Global_active_power'].astype('float')
data = data.dropna(subset = ['Global_active_power'])
data = data['Global_active_power'].values.reshape(-1,1)

scalar = MinMaxScaler()
scaled_data = scalar.fit_transform(data)

/tmp/ipykernel_17/1640542614.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('/kaggle/input/240000-household-electricity-consumption-records/household_power_consumption.csv', parse_dates=['Date'], index_col='Date')


In [5]:
def sequence_data(data,seq_length):
    x,y = [],[]
    for i in range(len(data) - seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(x),np.array(y)

seq_length = 24
X,y = sequence_data(scaled_data,seq_length)
        

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,Flatten,Dense

In [8]:
model = Sequential([
    Conv1D(64,kernel_size = 3,activation = 'relu',input_shape =(seq_length,1)),
    Conv1D(32,kernel_size = 3,activation = 'relu'),

    Flatten(),
    Dense(32,activation = 'relu'),
    Dense(1)
])

model.compile(optimizer ='adam', loss = 'mse',metrics = ['Accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.fit(X_train,y_train,epochs = 10, batch_size = 32,validation_data = (X_test,y_test))

Epoch 1/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - Accuracy: 0.0000e+00 - loss: 0.0013 - val_Accuracy: 1.9467e-05 - val_loss: 7.8368e-04
Epoch 2/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - Accuracy: 2.5753e-06 - loss: 8.4254e-04 - val_Accuracy: 1.9467e-05 - val_loss: 8.5075e-04
Epoch 3/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - Accuracy: 3.7950e-07 - loss: 8.0821e-04 - val_Accuracy: 1.9467e-05 - val_loss: 7.8652e-04
Epoch 4/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - Accuracy: 1.3656e-06 - loss: 7.8102e-04 - val_Accuracy: 1.9467e-05 - val_loss: 7.6305e-04
Epoch 5/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - Accuracy: 6.3252e-06 - loss: 7.6079e-04 - val_Accuracy: 1.9467e-05 - val_loss: 7.1183e-04
Epoch 6/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - Accuracy: 1.1287e-06 - loss: 7.5391e-04 - val_Accuracy: 1.9467e-05 - val_loss: 7.4167e-04
Epoch 7/10
6422/6422 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - Accuracy: 7.9734e-06 - loss: 7.5220e-04 - val_Accuracy: 1.9467e-0

In [10]:
from sklearn.metrics import mean_absolute_error
y_pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))


1606/1606 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
MAE: 0.011496372098248806


In [11]:
from sklearn.metrics import mean_squared_error
print("MSE:", mean_squared_error(y_test, y_pred))


MSE: 0.0007415110629008684
